In [1]:

from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, auc, roc_auc_score 

train = ['尽快，尽快,完成，工作',
    '可，代开，各，行业，发票，sep',
    '可，开，发票，请联系，sep',
    '明天，一起，约个饭，sep',
    '完成，的，非常，棒',
    '项目，效果，很好，sep',
    '有，发票，薇，sep',
    '有，开票，微，sep，有',
    '账单，请，查收',
    '正规，增值税，发票，sep']
label=[0,1,1,0,0,0,1,1,0,1]
test  = ['代开,增值税,发票,sep']

In [2]:

# 实例化
vectorizer_word = CountVectorizer(max_features=50,                              
                                 token_pattern=r"(?u)\b\w+\b",                   
                                 min_df=1,                       
                                 analyzer='word', 
                                 #ngram_range=(2,5) 
                                 ) 
#训练进行
vectorizer_word.fit(train)

#词转换成CountVectorizer向量
tfidf_train = vectorizer_word.transform(train) 
tfidf_test  = vectorizer_word.transform(test) 

In [11]:
len(vectorizer_word.vocabulary_)

29

In [9]:
# 数据处理
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 200) # 设置打印宽度(**重要**)

In [12]:
X = pd.DataFrame(tfidf_train.toarray()) # 对大于1的数据全部转换成1


X[X>1]=1

In [13]:
# 重命名÷


# 对字典进行反转
dic = {}
for k,v in vectorizer_word.vocabulary_.items():
    dic[v] = k
    
# 对数据集进行命名
X.columns = [dic[i] for i in list(X.columns)]

In [14]:

X

,sep,一起,代开,发票,可,各,增值税,完成,尽快,工作,开,开票,很好,微,效果,明天,有,查收,棒,正规,的,约个饭,薇,行业,请,请联系,账单,非常,项目
0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
5,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0
9,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [15]:
# 构建决策树规则提取函数
def Con_var_Get_Rules(clf,X):
    n_nodes = clf.tree_.node_count
    children_left = clf.tree_.children_left
    children_right = clf.tree_.children_right
    feature = clf.tree_.feature
    threshold = clf.tree_.threshold
    value = clf.tree_.value

    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves  = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, 0)]

    while len(stack) > 0:

        node_id, depth = stack.pop()
        node_depth[node_id] = depth

        is_split_node = children_left[node_id] != children_right[node_id]

        if is_split_node:
            stack.append((children_left[node_id],  depth+1))
            stack.append((children_right[node_id], depth+1))
        else:
            is_leaves[node_id] = True  
    feature_name = [
            X.columns[i] if i != _tree.TREE_UNDEFINED else "undefined!"
            for i in clf.tree_.feature]

    ways  = []
    depth = []
    feat = []
    nodes = []
    rules = []
    for i in range(n_nodes):   
        if  is_leaves[i]: 
            while depth[-1] >= node_depth[i]:
                depth.pop()
                ways.pop()    
                feat.pop()
                nodes.pop()
            if children_left[i-1]==i:#当前节点是上一个节点的左节点，则是小于
                a='{f}<={th}'.format(f=feat[-1],th=round(threshold[nodes[-1]],4))
                ways[-1]=a              
                last =' & '.join(ways)+':'+str(value[i][0][0])+':'+str(value[i][0][1])
                rules.append(last)
            else:
                a='{f}>{th}'.format(f=feat[-1],th=round(threshold[nodes[-1]],4))
                ways[-1]=a
                last = ' & '.join(ways)+':'+str(value[i][0][0])+':'+str(value[i][0][1])
                rules.append(last)

        else: #不是叶子节点 入栈
            if i==0:
                ways.append(round(threshold[i],4))
                depth.append(node_depth[i])
                feat.append(feature_name[i])
                nodes.append(i)             
            else: 
                while depth[-1] >= node_depth[i]:
                    depth.pop()
                    ways.pop()
                    feat.pop()
                    nodes.pop()
                if i==children_left[nodes[-1]]:
                    w='{f}<={th}'.format(f=feat[-1],th=round(threshold[nodes[-1]],4))
                else:
                    w='{f}>{th}'.format(f=feat[-1],th=round(threshold[nodes[-1]],4))              
                ways[-1] = w  
                ways.append(round(threshold[i],4))
                depth.append(node_depth[i]) 
                feat.append(feature_name[i])
                nodes.append(i)
    return rules

In [16]:

from sklearn import tree
import numpy as np
from sklearn.tree import _tree

# 构建决策树
clf = tree.DecisionTreeClassifier(
     max_depth=10,
     )
clf = clf.fit(X, label)

# 规则提取
Rules = Con_var_Get_Rules(clf,X)

In [17]:
Rules

['发票<=0.5 & 开票<=0.5:5.0:0.0', '发票<=0.5 & 开票>0.5:0.0:1.0', '发票>0.5:0.0:4.0']

In [19]:
# 结果格式整理，整理成合格的表格形式
df = pd.DataFrame(Rules)
df.columns = ['allrules']
df['Rules']    = df['allrules'].str.split(':').str.get(0)
df['good']     = df['allrules'].str.split(':').str.get(1).astype(float)
df['bad']      = df['allrules'].str.split(':').str.get(2).astype(float)
df['all']      = df['bad']+df['good']
df['bad_rate'] = df['bad']/df['all']

In [20]:

df

,allrules,Rules,good,bad,all,bad_rate
0,发票<=0.5 & 开票<=0.5:5.0:0.0,发票<=0.5 & 开票<=0.5,5.0,0.0,5.0,0.0
1,发票<=0.5 & 开票>0.5:0.0:1.0,发票<=0.5 & 开票>0.5,0.0,1.0,1.0,1.0
2,发票>0.5:0.0:4.0,发票>0.5,0.0,4.0,4.0,1.0
